In [23]:
from my_module.task_assigner.sub_functions import *

In [24]:
from main import get_data_task_assigner

In [44]:
data = get_data_task_assigner(access_token="Bearer eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJydXRUeHJ5dlltOUVZcGhpRjRxak45ajFvTktLNnU4YUhuM1QySFFSUU5FIn0.eyJleHAiOjE2NjM2MDE4NzMsImlhdCI6MTY2MzU4Mzg3MywiYXV0aF90aW1lIjoxNjYzNTgyNTIzLCJqdGkiOiI4YzhlMzhmOS1iYmM3LTQzODgtYTQ0Yi1hYjUwZjhjZTNmMzciLCJpc3MiOiJodHRwczovL2RldmVsb3BtZW50LmF1dGgud2FuZGVlZC5jb20vYXV0aC9yZWFsbXMvd2FuZGVlZC1yZWFsbSIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiIzMWZmODY3NC1iOGZhLTQyMmYtYWM3NC02YzFjZGI2YTUwZGUiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJ3YW5kZWVkLWNsaWVudCIsIm5vbmNlIjoiNmI0ZGVjMTAtY2Q2ZS00Y2Y1LTkzMTQtMmU2MDE5ZGEzY2UwIiwic2Vzc2lvbl9zdGF0ZSI6ImJiYmU4NjhjLWQyYTUtNDA0My1iYjVjLTQ3OWVkNGVmNjU3ZiIsImFjciI6IjAiLCJhbGxvd2VkLW9yaWdpbnMiOlsiaHR0cHM6Ly8qLndhbmRlZWQuY29tLyoiLCJodHRwczovLyouYXBpLndhbmRlZWQuY29tLyoiLCIqIiwiaHR0cHM6Ly8qLmFkbWluLndhbmRlZWQuY29tLyoiLCJodHRwczovLyouYXV0aC53YW5kZWVkLmNvbSJdLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsiZGVmYXVsdC1yb2xlcy13YW5kZWVkLXJlYWxtIiwib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFjY291bnQiLCJtYW5hZ2UtYWNjb3VudC1saW5rcyIsInZpZXctcHJvZmlsZSJdfX0sInNjb3BlIjoib3BlbmlkIGVtYWlsIHByb2ZpbGUiLCJzaWQiOiJiYmJlODY4Yy1kMmE1LTQwNDMtYmI1Yy00NzllZDRlZjY1N2YiLCJ1dGxfc3BrdXRpbGlzYXRldXIiOjEsInV0bF91dGlsaXNhdGV1cl9yb2xlcyI6IlsxXSIsInV0bF9jcHJlbm9tIjoiR2F5bG9yZCIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJ1dGxfc2FwcGFydGVuYW5jZSI6WzIsMzhdLCJ1dGxfY25vbSI6IlBldGl0IiwicHJlZmVycmVkX3VzZXJuYW1lIjoiZ2F5bG9yZC5wZXRpdEB0ZWFtYmVyLmZyIiwibG9jYWxlIjoiZnIiLCJnaXZlbl9uYW1lIjoiR2F5bG9yZCIsInV0bF9zZmtpbnN0YW5jZSI6MSwidXRsX3NhZG1pbmlzdHJlciI6W10sIm5hbWUiOiJHYXlsb3JkIFBldGl0IiwidXRsX3Nkcm9pdHNhY2NlcyI6WzYwLDMsNCw1LDE1LDI1LDI2LDMzLDM0LDM1LDM2LDM3LDQwLDQxLDQ0LDQ1LDQ2LDQ3XSwiZmFtaWx5X25hbWUiOiJQZXRpdCIsImVtYWlsIjoiZ2F5bG9yZC5wZXRpdEB0ZWFtYmVyLmZyIiwidXNlcl9ncm91cHMiOlsyLDM4LDYwLDMsNCw1LDE1LDI1LDI2LDMzLDM0LDM1LDM2LDM3LDQwLDQxLDQ0LDQ1LDQ2LDQ3XX0.q7k5-8CkwgfoOQWJIJ2CeBcIoMI_r7Ks2W-R8_80h6ef-zZ-2J60Ed6wZePs3j1NQuHggvWvS2tQ__YQMDQxKmgMiH0GEx_epof8DVU4UvfupmZ8vRwfbWzoPmBL16UqkNOb0ZOBdJkMFPnSjeCIMUHPzH2YH93IQi3bQ1FKWw4DHGMe7gBtQmbtfCqNPAFUH4XACXYK1Fxt3rzMqIqE_-5nAlJ_jbSjoBdsgURSyPbLxL12DlHrJsjlANtTBeacPxQa1xRhs7TlSQK_NwfBd0oUNOV32jLbu5qcOqlbOrrIg_VW0ob2tWAG1nA7dehHwW7036XA_yHYidMPGhojbQ",
                      datein_isoformat="2022-07-01T06:00:00.000Z",
                      dateout_isoformat="2022-09-30T06:00:00.000Z")

In [45]:
df_prj, df_cmp, df_tsk, df_dsp = split_data(data)

In [46]:
df_tsk

""


In [ ]:
def check_coherence_data(df_prj, df_cmp, df_tsk, df_dsp):
    if len(df_cmp)== 0:
        raise HTTPException(status_code=422,
            detail= "Pas de matrice de compétence disponible.")
    if len(df_tsk) == 0:
        raise HTTPException(status_code=422,
            detail= "Pas de tâches à assigner.")
        